## IMPORTS and UTILS

In [1]:
print("hi")

import os
import pandas as pd
import subprocess
import re
import numpy as np



import matplotlib.pyplot as plt
import seaborn as sns

import os

from robust_evaluation_tools.robust_utils import remove_covariates_effects, get_metrics, get_complete_combination
from robust_evaluation_tools.robust_harmonization import fit, apply, QC, compare_with_camcan, visualize_harmonization
from robust_evaluation_tools.robust_analysis import get_distribution_properties

test_metric = 'fa'
test_bundle = 'mni_AC'

harmonizartion_method = 'classic'
robust = 'No'
rwp = False
hc = False

HC_LABEL = 'HC'

metrics = get_metrics()

MAINFOLDER = f"RESULTS/FAKE_SICK/{test_metric}"
os.makedirs(MAINFOLDER, exist_ok=True)
# Read the camCan md file
camCan_file_test = os.path.join('DONNES','CamCAN', f"CamCAN.{test_metric}.raw.csv.gz")
data_dir = os.path.join('DONNES', "COMPILATIONS")
os.makedirs(data_dir, exist_ok=True)

main_test_file = os.path.join(data_dir,'FAKE_TEST', f"FAKE_CamCAN_test.{test_metric}.raw.csv.gz")

biases = [0.5, 1,-1, 2]


hi


In [2]:
def plot_df(df, bundle, metric):
    disease = df[df['disease'] != HC_LABEL]['disease'].unique()[0]
    df = df[df['bundle'] == bundle]
    plt.figure(figsize=(10, 6))
    plt.scatter(df[df['disease'] == HC_LABEL]['age'], df[df['disease'] == HC_LABEL]['mean'], color='blue', label='HC', alpha=0.5)
    plt.scatter(df[df['disease'] != HC_LABEL]['age'], df[df['disease'] != HC_LABEL]['mean'], color='red', label=f'{disease}', alpha=0.5)

    plt.title(f'Distribution of {disease} in metric {metric} for bundle: {bundle} (mean)')
    plt.xlabel('Age')
    plt.ylabel('Mean')
    plt.legend()
    plt.show()

    # Scatter plot with mean_no_cov
    plt.figure(figsize=(10, 6))
    plt.scatter(df[df['disease'] == HC_LABEL]['age'], df[df['disease'] == HC_LABEL]['mean_no_cov'], color='blue', label='HC', alpha=0.5)
    plt.scatter(df[df['disease'] != HC_LABEL]['age'], df[df['disease'] != HC_LABEL]['mean_no_cov'], color='red', label=f'{disease}', alpha=0.5)

    plt.title(f'Distribution of {disease} in metric {metric} for bundle: {bundle} (mean_no_cov)')
    plt.xlabel('Age')
    plt.ylabel('Mean_no_cov')
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 6))
    sns.kdeplot(df[df['disease'] == HC_LABEL]['mean_no_cov'], color='blue', label='HC', fill=True)
    sns.kdeplot(df[df['disease'] != HC_LABEL]['mean_no_cov'], color='red', label=f'{disease}', fill=True)
    plt.title(f'Distribution of {disease} in metric {metric} for bundle: {bundle}')
    plt.xlabel('mean_no_cov')
    plt.ylabel('Frequency')
    plt.legend()

In [3]:
camCan_copies = []
camCan_files_copies = []
for metric in metrics:
    print(f"-------------------- {metric} ----------------------")
    camCan_file = os.path.join('DONNES','CamCAN', f"CamCAN.{metric}.raw.csv.gz")
    camCan = pd.read_csv(camCan_file)
    # Randomly select 100 unique 'sid' values

    biaised_camCan = camCan.copy()
    biaised_camCan['mean'] += 0.000
    biaised_camCan = remove_covariates_effects(biaised_camCan)

    test_sids = np.random.choice(biaised_camCan['sid'].unique(), 100, replace=False)

    # Create the test dataframe with the selected 'sid' values
    test = biaised_camCan[biaised_camCan['sid'].isin(test_sids)]
    test_dir = os.path.join(data_dir,'FAKE_TEST')
    os.makedirs(test_dir, exist_ok=True)
    test_file = os.path.join(test_dir, f"FAKE_CamCAN_test.{metric}.raw.csv.gz")
    test.to_csv(test_file, index=False)

    # Remove the selected 'sid' values from the original camCan dataframe
    biaised_camCan = biaised_camCan[~biaised_camCan['sid'].isin(test_sids)]

    for bias in biases:
        print(f"-------------------- {bias} ----------------------")
        dir = os.path.join(data_dir, f"SYN_{bias}")
        os.makedirs(dir, exist_ok=True)
        camCan_copy = biaised_camCan.copy()
        for bundle in camCan_copy['bundle'].unique():
            camCan_copy.loc[camCan_copy['bundle'] == bundle, 'mean'] += bias * camCan_copy.loc[camCan_copy['bundle'] == bundle, 'mean_no_cov'].std()
        camCan_copy['disease'] = f'SYN_{bias}'
        camCan_copy['sid'] = camCan_copy['sid'].astype(str) + f'_{bias}'
        camCan_copy = pd.concat([biaised_camCan, camCan_copy])
        camCan_copy = remove_covariates_effects(camCan_copy)
        for bundle in camCan_copy['bundle'].unique():
            subset_all = camCan_copy[camCan_copy['bundle'] == bundle]
            hc_data = subset_all[subset_all['disease'] == HC_LABEL]['mean_no_cov'].dropna()
            non_hc_data = subset_all[subset_all['disease'] != HC_LABEL]['mean_no_cov'].dropna()
            mean_hc = np.mean(hc_data)
            mean_non_hc = np.mean(non_hc_data)
            euclidean_dist = abs(mean_hc - mean_non_hc)    
            print(euclidean_dist/np.std(hc_data))

        camCan_copy['site'] = f'SYN_{bias}_compialtion'
        file = os.path.join(dir, f"SYN_{bias}_combination.{metric}.raw.csv.gz")
        camCan_copy.to_csv(file, index=False)
        if metric == test_metric:
            camCan_copies.append((bias,camCan_copy))
            camCan_files_copies.append(file)


-------------------- ad ----------------------
-------------------- 0.5 ----------------------
0.5009922895178429
0.5012111878019758
0.5016572408963856
0.501075801069824
0.5013454312748138
0.5011134633320552
0.5014424477415967
0.5014390478479271
0.5012708554173414
0.5012732566354842
0.5014321502328803
0.5011547264979
0.5013935395886441
0.501538228127465
0.5012711170520213
0.501022667919792
0.5015286915108058
0.5009729650990259
0.5010362564287407
0.5014525166383158
0.501132503030771
0.5011239177356055
0.5018062176247374
0.5011092399318038
0.5009232843077224
0.5007728602180597
0.5007924436018864
0.5019574297577476
0.5013086982227409
0.5013716292300427
0.5009653542964191
0.501737098134489
0.5010429596607673
0.5013583212380949
0.5013094707451665
0.5008388105225562
0.5017275514702918
0.5011557158437028
0.5007744422266392
0.500769803967336
0.5014183322543458
0.5012326218311633
0.5017789347808068
0.5011769020341919
0.5010616930375582
-------------------- 1 ----------------------
1.00198457903

In [4]:
raw_directory = os.path.join('DONNES', 'COMPILATIONS')
for bias in biases:
        disease = f'SYN_{bias}'
        print('--------------------', disease, '-----------------------')
        combination = get_complete_combination(os.path.join(raw_directory, disease), f'{disease}_combination*.csv.gz', False, False)
        combination['metric_bundle'] = combination['metric'] + "_" + combination['bundle']
        combination.to_csv(os.path.join(raw_directory, f'{disease}_combination_all_metrics_CamCAN.csv.gz'), index=False)
        combination.to_csv(os.path.join(raw_directory, f'{disease}_combination_all_metrics.csv.gz'), index=False)

-------------------- SYN_0.5 -----------------------
-------------------- SYN_1 -----------------------
-------------------- SYN_-1 -----------------------
-------------------- SYN_2 -----------------------


In [ ]:
# Create an array of biases
camCan_files_copies = []
for bias in biases:
    dir = os.path.join(data_dir, f"SYN_{bias}")
    file = os.path.join(dir, f"SYN_{bias}_combination.{test_metric}.raw.csv.gz")
    camCan_files_copies.append(file)
    plot_df(pd.read_csv(file), test_bundle, test_metric)

In [ ]:
dir = os.path.join(MAINFOLDER, 'FULL_SITE')
os.makedirs(dir, exist_ok=True)
for copy_filename in camCan_files_copies:
    output_model_filename_sick_1 = fit(copy_filename, camCan_file_test, metric, harmonizartion_method, robust, rwp, dir, hc,)
    # Apply the model
    output_filename_sick_1 = apply(copy_filename, output_model_filename_sick_1, metric, harmonizartion_method, robust, rwp, dir)

    visualize_harmonization(copy_filename, output_filename_sick_1, camCan_file_test, dir, test_bundle)

In [ ]:
def apply_fixed_number_of_sick(df,test_file, diff_sick, total ,proportion, metric, bundle, harmonizartion_method, robust, rwp, directory, hc, camCan_file, index):
    # Create a copy of the dataframe
    df_copy = df.copy()
    num_sick = int(proportion * total)
    num_healthy = total - num_sick
    
    # Separate healthy and sick samples
    healthy_samples = df_copy[df_copy['disease'] == HC_LABEL]
    sick_samples = df_copy[df_copy['disease'] != HC_LABEL] 
    
    selected_healthy_sids = np.random.choice(healthy_samples['sid'].unique(), num_healthy, replace=False)
    selected_sick_sids = np.random.choice(sick_samples['sid'].unique(), num_sick, replace=False)
    
    # Filter the dataframe to include only the selected samples
    df_selected = df_copy[df_copy['sid'].isin(selected_healthy_sids) | df_copy['sid'].isin(selected_sick_sids)]
    # Change the site of the selected samples
    df_selected['site'] = f'CamCAN_sick-{proportion}-{index}'
    
    # Save the modified dataframe to a file
    file_path = os.path.join(directory, f"CamCAN_sick_{proportion}_{index}.{metric}.raw.csv.gz")
    df_selected.to_csv(file_path, index=False)
    
    # Apply the fit function
    output_model_filename = fit(file_path, camCan_file, metric, harmonizartion_method, robust, rwp, directory, hc)
    
    # Apply the model
    output_filename = apply(file_path, output_model_filename, metric, harmonizartion_method, robust, rwp, directory)

    output_filename_test = apply(test_file, output_model_filename, metric, harmonizartion_method, robust, rwp, directory)


    res = compare_with_camcan(pd.read_csv(output_filename))
    res['site'] = f'CamCAN_sick{diff_sick}_{proportion}_{index}'
    res['disease_ratio'] = proportion
    res['index'] = index
    res['num_patients'] = total
    res['metric'] = metric
    res['method'] = 'train'

    res_test = compare_with_camcan(pd.read_csv(output_filename_test))
    res_test['site'] = f'CamCAN_sick{diff_sick}_{proportion}_{index}'
    res_test['disease_ratio'] = proportion
    res_test['index'] = index
    res_test['num_patients'] = total
    res_test['metric'] = metric
    res_test['method'] = 'test'

    distr_metrics = get_distribution_properties(df_selected)
    distr_metrics['site'] = f'CamCAN_sick{diff_sick}_{proportion}_{index}'
    distr_metrics['disease_ratio'] = proportion
    distr_metrics['index'] = index
    distr_metrics['num_patients'] = total
    distr_metrics['metric'] = metric
    distr_metrics['method'] = 'train'
    
    
    # Visualize the harmonization
    visualize_harmonization(file_path, output_filename, camCan_file, directory, bundle)

    return pd.concat([res, res_test]), distr_metrics



# Define the number of sick samples to test
total = 200

# Apply the function to the biaised_camCan dataframe
proportions = [0.0, 0.2, 0.3, 0.4, 0.6, 0.8]
total_mae = pd.DataFrame()
total_distr_metrics = pd.DataFrame()

for biais, copie in camCan_copies:
    dir = os.path.join(MAINFOLDER, "PROPORTION", f"CamCAN_sick_{str(biais)}")
    os.makedirs(dir, exist_ok=True)
    for proportion in proportions:
        for index in range(5):
            mae, distr_metrics = apply_fixed_number_of_sick(copie, main_test_file, biais, total, proportion, test_metric, test_bundle, harmonizartion_method, robust, rwp, dir, hc, camCan_file_test, index)
            mae['diff_sick'] = str(biais)
            distr_metrics['diff_sick'] = str(biais)
            total_mae = pd.concat([total_mae, mae])
            total_distr_metrics = pd.concat([total_distr_metrics, distr_metrics])

In [ ]:
total_mae

In [ ]:
total_distr_metrics

In [ ]:
# CREATION BOX PLOT POUR MEA
def plot_mea(df, sample_size, diff_sick, metric, directory):
    """
    Crée un graphique pour chaque bundle dans le DataFrame donné.
    L'axe des X représente le nombre de patients et l'axe des Y représente la moyenne de la colonne du bundle.
    La courbe inclut une zone indiquant l'écart-type (std).

    Parameters:
    df (pd.DataFrame): Le DataFrame contenant les données.
    bundle_column (str): Le nom de la colonne du bundle à utiliser pour le graphique.
    """
    directory = os.path.join(directory, "MEA_PLOTS", str(diff_sick), metric, str(sample_size))
    df = df[df['num_patients'] == sample_size]
    df = df[df['diff_sick'] == diff_sick]
    df = df[df['metric'] == metric]
    os.makedirs(directory, exist_ok=True)
    
    for bundle_column in df.columns:
        if bundle_column in ['site','num_patients','disease_ratio', 'metric', 'diff_sick', 'method', 'index']:
            continue # Skip non-numeric columns
        bundle_df = df[[bundle_column, 'site' ,'index','disease_ratio','diff_sick', 'method']].copy()
        methods = ["train", "test"]
        colors = ['green', 'red']
        
        plt.figure(figsize=(10, 5))
        width = 0.2  # the width of the bars
        x = np.arange(len(bundle_df['disease_ratio'].unique()))  # the label locations
        fig, ax = plt.subplots(figsize=(10, 5))

        for i, (method, color) in enumerate(zip(methods, colors)):
            method_df = bundle_df[bundle_df['method'] == method]
            data = [method_df[method_df['disease_ratio'] == patients][bundle_column].values 
                    for patients in bundle_df['disease_ratio'].unique()]
            
            # Ensure there is data for each num_patients
            if any(len(d) > 0 for d in data):
                positions = x + i * width  # Shift positions for each method
                ax.boxplot(data, positions=positions, widths=0.15, patch_artist=True, 
                        boxprops=dict(facecolor=color, color=color),
                        medianprops=dict(color='black'))
                
        ax.set_xlabel('Prct de patients malades')
        ax.set_ylabel('MAE')
        ax.set_title(f'MAE Maladie: {diff_sick} bundle: {bundle_column}, metrique : {metric} avec {sample_size} patients')
        ax.set_xticks(x + width * (len(methods) - 1) / 2)
        ax.set_xticklabels(bundle_df['disease_ratio'].unique())
        ax.legend(handles=[plt.Line2D([0], [0], color=color, lw=2, label=f'Method: {method}') for method, color in zip(methods, colors)])
        plt.savefig(os.path.join(directory, f'{bundle_column}_boxplot.png'))
        plt.close()

sample_sizes = [200]

for diff_sick in total_mae['diff_sick'].unique():
    for sample_size in sample_sizes:
        plot_mea(total_mae, sample_size, diff_sick, test_metric, MAINFOLDER)

In [ ]:
# CREATION BOX PLOT POUR property
def plot_properties(df, property, sample_size, diff_sick, metric, directory):
    """
    Crée un graphique pour chaque bundle dans le DataFrame donné.
    L'axe des X représente le nombre de patients et l'axe des Y représente la moyenne de la colonne du bundle.
    La courbe inclut une zone indiquant l'écart-type (std).

    Parameters:
    df (pd.DataFrame): Le DataFrame contenant les données.
    bundle_column (str): Le nom de la colonne du bundle à utiliser pour le graphique.
    """
    directory = os.path.join(directory, "PROPERTIES_PLOTS", str(diff_sick), property, metric, str(sample_size))
    df = df[df['property'] == property]
    df = df[df['num_patients'] == sample_size]
    df = df[df['diff_sick'] == diff_sick]
    df = df[df['metric'] == metric]
    os.makedirs(directory, exist_ok=True)
    
    for bundle_column in df.columns:
        if bundle_column in ['site','num_patients','disease_ratio', 'metric', 'diff_sick', 'method', 'index', 'property']:
            continue # Skip non-numeric columns
        bundle_df = df[[bundle_column, 'site' ,'index','disease_ratio','diff_sick', 'method']].copy()
        methods = ["train"]
        colors = ['green']
        
        plt.figure(figsize=(10, 5))
        width = 0.2  # the width of the bars
        x = np.arange(len(bundle_df['disease_ratio'].unique()))  # the label locations
        fig, ax = plt.subplots(figsize=(10, 5))

        for i, (method, color) in enumerate(zip(methods, colors)):
            method_df = bundle_df[bundle_df['method'] == method]
            data = [method_df[method_df['disease_ratio'] == patients][bundle_column].values 
                    for patients in bundle_df['disease_ratio'].unique()]
            
            # Ensure there is data for each num_patients
            if any(len(d) > 0 for d in data):
                positions = x + i * width  # Shift positions for each method
                ax.boxplot(data, positions=positions, widths=0.15, patch_artist=True, 
                        boxprops=dict(facecolor=color, color=color),
                        medianprops=dict(color='black'))
                
        ax.set_xlabel('Prct de patients malades')
        ax.set_ylabel(property)
        ax.set_title(f'{property} Maladie: {diff_sick} bundle: {bundle_column}, metrique : {metric} avec {sample_size} patients')
        ax.set_xticks(x + width * (len(methods) - 1) / 2)
        ax.set_xticklabels(bundle_df['disease_ratio'].unique())
        ax.legend(handles=[plt.Line2D([0], [0], color=color, lw=2, label=f'Method: {method}') for method, color in zip(methods, colors)])
        plt.savefig(os.path.join(directory, f'{bundle_column}_boxplot.png'))
        plt.close()

sample_sizes = [200]
total_distr_metrics
#Stotal_distr_metrics['diff_sick'] = [0] * 360 + [1] * 360 + [2] * 360
for diff_sick in total_distr_metrics['diff_sick'].unique():
    for property in total_distr_metrics['property'].unique():
        for sample_size in sample_sizes:
            plot_properties(total_distr_metrics, property, sample_size, diff_sick, test_metric, MAINFOLDER)

In [ ]:
combined_df = pd.merge(total_distr_metrics, total_mae, on=['site', 'metric', 'disease_ratio', 'index', 'diff_sick'], suffixes=('_distr', '_mae'))
combined_df


In [ ]:
correlation_df = pd.DataFrame(columns=['bundle', 'diff_sick', 'property', 'correlation'])
# Create scatter plots for each bundle, diff_sick, and property
for property in combined_df['property'].unique():
    df_property_filtered = combined_df[combined_df['property'] == property]
    for diff_sick in df_property_filtered['diff_sick'].unique():
        df_filtered = df_property_filtered[df_property_filtered['diff_sick'] == diff_sick]
        for bundle in total_mae.columns:
            if bundle in ['site', 'metric', 'disease_ratio', 'index', 'diff_sick', 'method', 'num_patients', 'property']:
                continue  # Skip non-numeric columns

            plt.figure(figsize=(10, 6))
            for method, color in zip(['train', 'test'], ['red', 'blue']):
                method_filtered = df_filtered[df_filtered['method_mae'] == method]
                plt.scatter(method_filtered[f'{bundle}_distr'], method_filtered[f'{bundle}_mae'], alpha=0.5, label=method, color=color)
            
            # Calculate correlation
            correlation = df_filtered[[f'{bundle}_distr', f'{bundle}_mae']].corr().iloc[0, 1]
            plt.title(f'Scatter plot for {bundle} (diff_sick: {diff_sick}, property: {property})\nCorrelation: {correlation:.2f}')
            plt.xlabel(f'{bundle} {property}')
            plt.ylabel(f'{bundle} MAE')
            plt.legend()
            plt.grid(True)
            
            # Create directory path
            directory = os.path.join(MAINFOLDER, 'PROPERTY_SCATTER' ,  f'{property}', f'diff_sick_{diff_sick}')
            os.makedirs(directory, exist_ok=True)
            
            # Save the plot
            plt.savefig(os.path.join(directory, f'scatter_plot_{bundle}.png'))
            plt.close()
            
            # Store correlation in a new dataframe
            new_row = pd.DataFrame({'bundle': [bundle], 'diff_sick': [diff_sick], 'property': [property], 'correlation': [correlation]})
            correlation_df = pd.concat([correlation_df, new_row], ignore_index=True)

In [ ]:
correlation_df['abs_correlation'] = correlation_df['correlation'].abs()
correlation_df = correlation_df.sort_values(by='abs_correlation', ascending=False)
correlation_df

In [16]:
correlation_df_all = pd.DataFrame(columns=['bundle', 'property', 'correlation'])
# Create scatter plots for each bundle, diff_sick, and property
for property in combined_df['property'].unique():
    df_filtered = combined_df[combined_df['property'] == property]
    for bundle in total_mae.columns:
        if bundle in ['site', 'metric', 'disease_ratio', 'index', 'diff_sick', 'method', 'num_patients', 'property']:
            continue  # Skip non-numeric columns

        plt.figure(figsize=(10, 6))
        for method, color in zip(['train', 'test'], ['red', 'blue']):
            method_filtered = df_filtered[df_filtered['method_mae'] == method]
            plt.scatter(method_filtered[f'{bundle}_distr'], method_filtered[f'{bundle}_mae'], alpha=0.5, label=method, color=color)
        
        # Calculate correlation
        correlation = df_filtered[[f'{bundle}_distr', f'{bundle}_mae']].corr().iloc[0, 1]
        plt.title(f'Scatter plot for {bundle} , property: {property})\nCorrelation: {correlation:.2f}')
        plt.xlabel(f'{bundle} {property}')
        plt.ylabel(f'{bundle} MAE')
        plt.legend()
        plt.grid(True)
        
        # Create directory path
        directory = os.path.join(MAINFOLDER, 'PROPERTY_SCATTER_ALL' ,  f'{property}')
        os.makedirs(directory, exist_ok=True)
        
        # Save the plot
        plt.savefig(os.path.join(directory, f'scatter_plot_{bundle}.png'))
        plt.close()
        
        # Store correlation in a new dataframe
        new_row = pd.DataFrame({'bundle': [bundle], 'property': [property], 'correlation': [correlation]})
        correlation_df_all = pd.concat([correlation_df_all, new_row], ignore_index=True)

/tmp/ipykernel_1651398/3454534488.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  correlation_df_all = pd.concat([correlation_df_all, new_row], ignore_index=True)


In [19]:
correlation_df_all['abs_correlation'] = correlation_df_all['correlation'].abs()
correlation_df_all = correlation_df_all.sort_values(by='abs_correlation', ascending=False)
correlation_df_all

,bundle,property,correlation,abs_correlation
77,mni_SCP,mean_median_shift,0.544429,0.544429
188,mni_ICP_L,left_right_mean_diff,0.502905,0.502905
120,mni_SCP,kurtosis,-0.487238,0.487238
206,mni_SCP,left_right_mean_diff,0.486205,0.486205
66,mni_MCP,mean_median_shift,0.484466,0.484466
...,...,...,...,...
7,mni_CC_ForcepsMajor,skewness,-0.005501,0.005501
154,mni_ML_R,bowley_skewness,-0.004927,0.004927
136,mni_CC_ForcepsMajor,bowley_skewness,0.004706,0.004706
228,mni_FPT_L,left_right_std_ratio,-0.000286,0.000286


In [ ]:
correlation_df
